In [ ]:
# from pymystem3 import Mystem
# m = Mystem()

Выбрала pymorphy, а не mystem, потому что pymorphy позволяет выбирать из нескольких вариантов разбора (если автоматический не подошел). mystem предлагает неверный вариант вариант в 'Даша мыла яблоки'. pymorphy тоже, но можно выбрать нужный.

Сначала импортируем pymorphy и создадим экземпляр класса для анализатора. 

In [ ]:
import pymorphy2
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [ ]:
corpus = ["Мама мыла раму", "Даша мыла яблоки", "Даша очень любит маму"]

Напишем функцию для лемматизации элементов списка.

In [ ]:
def normalize(text):
    words = text.split()  # Разобьем на слова, потому что pymorphy работает с отдельными словами
    lemmas = list()  # В этот список будем складывать леммы
    for word in words:
        parse = morph.parse(word)  # Парсим каждое слово
        if len(parse) > 2:  # Отловим проблемное слово, где много вариантов разбора, в которых путается pymorphy  
            for p in parse:
                if 'VERB' in p.tag:  # Ищем тот вариант разбора, где глагол 'мыть'
                    lemmas.append(p.normal_form)
        else:
            lemmas.append(parse[0].normal_form)  # Во всех остальных случаях берем первый вариант разбора, он верный
    lem = ' '.join(lemmas)  # По условию задачи объединяем отдельные леммы в список
    return(lem)

Вызовем функцию для каждого элемента корпуса. Получим нужный результат

In [ ]:
corpus_lem = list()
for i in corpus:
    corpus_lem.append(normalize(i))
print(corpus_lem)

Делим полученный лемматизированный корпус на токены и создадим словарь:

In [ ]:
corpus_tokenized = [sentence.split() for sentence in corpus_lem]
corpus_tokenized

In [ ]:
word_indices = {}

for sentence in corpus_tokenized:
    for word in sentence:
        word_indices[word] = 0

Я костылем его заполнила значениями по порядку, может решение не изящное, но после плясок с пайморфи времени не осталось на продумывание изящности.

In [ ]:
v = 0
for k in word_indices.keys():
    word_indices[k] = v
    v += 1
word_indices  

Создадим словарь-гистограмму с частотностью слов.

In [ ]:
count_dictionary = {}

for sentence in corpus_tokenized:
    words = set(sentence)  # Убираем дубликаты
    for word in words:
        # Заполняем частотный словарь - количество документов, в которых встретилось слово 
        count_dictionary[word] = count_dictionary.get(word, 0) + 1
count_dictionary

В формуле idf, как помнится мы выяснили, применяется десятичный логарифм, а не натуральный. Вроде бы выбор десятичного или натурального логарифма на конечный результат не влияет, но я оставлю десятичный.

In [ ]:
from math import log10

In [ ]:
idf = {}
for k,v in count_dictionary.items():
    idf[k] = log10(len(corpus)/v)  # Формула для вычисления idf, данные берем из частотного словаря 
idf

Инициализируем векторы для каждого документа в корпусе, они пока заполнены нулевыми значениями.

In [ ]:
tf_idf = [[0] * len(idf) for i in range(len(corpus_tokenized))]

In [ ]:
from collections import Counter

In [ ]:
# "Пронумеруем" каждый список токенов в корпусе, чтобы потом заполнить соответствующие векторы
for n, sentence in enumerate(corpus_tokenized):
    words_counter = Counter(sentence)
    sentence_length = len(sentence)
    # Пройдем по каждому слову и вычислим для него tf-idf
    for word in words_counter:
        word_index = word_indices[word]
        word_tf_idf = (idf[word]/sentence_length)
        print(word, '-->', word_tf_idf) # Посмотрим tf-idf каждого слова
        # Заполним соответствующие ячейки векторов
        tf_idf[n][word_index] = word_tf_idf
tf_idf # Посмотрим на векторы

Теперь вычислим косинусное сходство, чтобы понять, какие предлождения ближе друг к другу. 

In [ ]:
from scipy.spatial.distance import cosine

In [ ]:
result1 = 1 - cosine(tf_idf[0], tf_idf[1])
result2 = 1 - cosine(tf_idf[1], tf_idf[2])
result3 = 1 - cosine(tf_idf[0], tf_idf[2])

print('Первое и второе: ', result1)
print('Второе и третье: ', result2)
print('Первое и третье: ', result3)

Ближе всего второе и третье, а также первое и третье, потому что более близки те векторы, у которых меньше косинусное расстояние.